<h1><b><i>RISCOS - BI</b></i></h1>


In [119]:
import pandas as pd

In [120]:
df_riscos = pd.read_excel('Planilha Monitoramento de Riscos_Modelo_RODOVIÁRIO_23_09_24.xlsx')

c:\Users\lucas.bruno\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [121]:
file_path = 'Planilha Monitoramento de Riscos_Modelo_RODOVIÁRIO_23_09_24.xlsx'

In [122]:
#CHECKPOINT
df_riscos_copy = df_riscos.copy()

In [123]:
df_riscos_copy = pd.read_excel(file_path, sheet_name='RISCOS AMBIENTAIS', header=2)


c:\Users\lucas.bruno\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [124]:
df_riscos_copy.columns

Index(['SETOR', 'UF', 'BR', 'Unnamed: 3', 'EMPREENDIMENTO',
       'EXECUTOR             (Grupo Controlador)',
       'DATA DE INÍCIO DA CONCESSÃO', 'ANO DA CONCESSÃO', 'INTERFERÊNCIAS',
       'INTERVENÇÃO AFETADA', 'LOCALIZAÇÃO', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'PER', 'LICENÇAS', 'Unnamed: 17',
       'Unnamed: 18', 'OBRAS', 'PROCESSO', 'Unnamed: 21', 'Unnamed: 22',
       'INTERFERÊNCIAS.1', 'CRITICIDADE', 'SITUAÇÃO GERAL',
       'COMENTÁRIOS GERAIS', 'Unnamed: 27', 'Unnamed: 28'],
      dtype='object')

In [125]:
df_riscos_copy = df_riscos_copy.rename(columns={'Unnamed: 3': 'Bin', 'Unnamed: 11': 'km(i)', 'Unnamed: 12': 'km(f)', 
                                                'Unnamed: 13': 'Ext.(km)', 'Unnamed: 14': 'FINANCEIRO(R$)',
                                                'Unnamed: 17': 'LI', 'Unnamed: 18': 'LO', 'Unnamed: 21': 'SISGLAF', 
                                                'Unnamed: 22': 'SEI', 'Unnamed: 27': 'Data do Atendimento', 
                                                'Unnamed: 28': 'NÍVEL DE ALERTA',
                                                'EXECUTOR             (Grupo Controlador)': 'EXECUTOR\n(Grupo Controlador)',
                                                'LOCALIZAÇÃO': 'Descrição', 
                                                'PER': 'PER\nANO(Execução)', 
                                                'LICENÇAS': 'LP', 
                                                'OBRAS': 'DATA DE INÍCIO', 'PROCESSO': 'SINAFLOR', 
                                                'INTERFERÊNCIAS.1': 'Risco Principal'})

In [126]:
# Agora, modifique as colunas 'km(i)' e 'km(f)' para incluir o '+'
for col in ['km(i)', 'km(f)']:
    df_riscos_copy[col] = df_riscos_copy[col].astype(str).str.replace('(\d{3})(\d+)', r'\1+\2', regex=True)

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_8868\1936325901.py:3: SyntaxWarning: invalid escape sequence '\d'
  df_riscos_copy[col] = df_riscos_copy[col].astype(str).str.replace('(\d{3})(\d+)', r'\1+\2', regex=True)


In [127]:
# Colunas que precisam ser preenchidas até encontrar '1' em 'BR'
colunas_para_preencher = ['EMPREENDIMENTO', 'EXECUTOR\n(Grupo Controlador)',
       'DATA DE INÍCIO DA CONCESSÃO', 'ANO DA CONCESSÃO']

# Iterar sobre as linhas do DataFrame
for coluna in colunas_para_preencher:
    valor_atual = None
    for idx, row in df_riscos_copy.iterrows():
        # Checa se 'BR' é '1' ou se a linha da coluna a ser preenchida não está vazia
        if row['BR'] == '1' or pd.notna(row[coluna]):
            valor_atual = row[coluna]
        df_riscos_copy.at[idx, coluna] = valor_atual

In [128]:
df_riscos_copy = df_riscos_copy.drop([0, 1, 2], axis = 0)
df_riscos_copy = df_riscos_copy.reset_index(drop=True)

In [129]:
df_riscos_copy = df_riscos_copy.loc[df_riscos_copy['BR'] != 1]

In [130]:
df_riscos_copy['BR'].unique()

array([101, 292, 386, 448, 364, 365, 153, '116 (Dutra)',
       '116 (Santos Dumont', nan, 116, 465, 493, 163, 230, '040', 392,
       393, 381, 376, 324, 526, 527, '050', 406, '060'], dtype=object)

In [131]:
df_riscos_copy = df_riscos_copy.reset_index(drop=True)

In [132]:
# Função para limpar e converter as datas para o formato datetime
def limpar_e_converter_data(data):
    # Tratar dados como string para evitar problemas com diferentes tipos
    data = str(data)
    # Substituir '//'' por '/'
    data = data.replace('//', '/')
    # Substituir '/' por '-'
    data = data.replace('/', '-')
    # Tentar converter para datetime
    return pd.to_datetime(data, errors='coerce', dayfirst=True)

# Aplicar a função no DataFrame
# Substitua 'df_riscos_copy' pelo nome do seu DataFrame e 'DATA DE INÍCIO DA CONCESSÃO' pelo nome da coluna que contém as datas
df_riscos_copy['DATA DE INÍCIO DA CONCESSÃO'] = df_riscos_copy['DATA DE INÍCIO DA CONCESSÃO'].apply(limpar_e_converter_data)

C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_8868\4202058614.py:10: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(data, errors='coerce', dayfirst=True)


In [133]:
# Converter colunas para 'category'
df_riscos_copy['SETOR'] = df_riscos_copy['SETOR'].astype('object')
df_riscos_copy['UF'] = df_riscos_copy['UF'].astype('category')
df_riscos_copy['INTERFERÊNCIAS'] = df_riscos_copy['INTERFERÊNCIAS'].astype('category')
df_riscos_copy['INTERVENÇÃO AFETADA'] = df_riscos_copy['INTERVENÇÃO AFETADA'].astype('category')
df_riscos_copy['EXECUTOR\n(Grupo Controlador)'] = df_riscos_copy['EXECUTOR\n(Grupo Controlador)'].astype('category')
#df_riscos_copy['PROCESSO'] = df_riscos_copy['PROCESSO'].astype('category')
df_riscos_copy['Risco Principal'] = df_riscos_copy['Risco Principal'].astype('category')
#df_riscos_copy['Licenças'] = df_riscos_copy['Licenças'].astype('category')
#df_riscos_copy['Valor_Licenças'] = df_riscos_copy['Valor_Licenças'].astype('category')
df_riscos_copy['CRITICIDADE'] = df_riscos_copy['CRITICIDADE'].astype('category')
df_riscos_copy['NÍVEL DE ALERTA'] = df_riscos_copy['NÍVEL DE ALERTA'].astype('category')

# Converter colunas para 'int'
df_riscos_copy['FINANCEIRO(R$)'] = df_riscos_copy['FINANCEIRO(R$)'].replace('[\$,]', '', regex=True).replace(',', '.', regex=True)
df_riscos_copy['FINANCEIRO(R$)'] = pd.to_numeric(df_riscos_copy['FINANCEIRO(R$)'], errors='coerce')

df_riscos_copy['ANO DA CONCESSÃO'] = pd.to_numeric(df_riscos_copy['ANO DA CONCESSÃO'], errors='coerce').fillna(0).astype(int)

# Converter colunas para 'float'
df_riscos_copy['Ext.(km)'] = pd.to_numeric(df_riscos_copy['Ext.(km)'], errors='coerce')

# Converter colunas para 'string'
df_riscos_copy['EMPREENDIMENTO'] = df_riscos_copy['EMPREENDIMENTO'].astype('string')
#df_riscos_copy['Nº PROCESSO'] = df_riscos_copy['Nº PROCESSO'].astype('string')
df_riscos_copy['Descrição'] = df_riscos_copy['Descrição'].astype('string')
df_riscos_copy['SITUAÇÃO GERAL'] = df_riscos_copy['SITUAÇÃO GERAL'].astype('string')  
df_riscos_copy['COMENTÁRIOS GERAIS'] = df_riscos_copy['COMENTÁRIOS GERAIS'].astype('string')

<>:15: SyntaxWarning: invalid escape sequence '\$'
<>:15: SyntaxWarning: invalid escape sequence '\$'
C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_8868\1743130282.py:15: SyntaxWarning: invalid escape sequence '\$'
  df_riscos_copy['FINANCEIRO(R$)'] = df_riscos_copy['FINANCEIRO(R$)'].replace('[\$,]', '', regex=True).replace(',', '.', regex=True)
C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_8868\1743130282.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_riscos_copy['FINANCEIRO(R$)'] = df_riscos_copy['FINANCEIRO(R$)'].replace('[\$,]', '', regex=True).replace(',', '.', regex=True)


In [134]:
df_riscos_copy.columns

Index(['SETOR', 'UF', 'BR', 'Bin', 'EMPREENDIMENTO',
       'EXECUTOR\n(Grupo Controlador)', 'DATA DE INÍCIO DA CONCESSÃO',
       'ANO DA CONCESSÃO', 'INTERFERÊNCIAS', 'INTERVENÇÃO AFETADA',
       'Descrição', 'km(i)', 'km(f)', 'Ext.(km)', 'FINANCEIRO(R$)',
       'PER\nANO(Execução)', 'LP', 'LI', 'LO', 'DATA DE INÍCIO', 'SINAFLOR',
       'SISGLAF', 'SEI', 'Risco Principal', 'CRITICIDADE', 'SITUAÇÃO GERAL',
       'COMENTÁRIOS GERAIS', 'Data do Atendimento', 'NÍVEL DE ALERTA'],
      dtype='object')

In [135]:
df_riscos_copy = df_riscos_copy[['SETOR', 'UF', 'BR', 'Bin', 'EMPREENDIMENTO',
       'EXECUTOR\n(Grupo Controlador)', 'DATA DE INÍCIO DA CONCESSÃO',
       'ANO DA CONCESSÃO', 'INTERFERÊNCIAS', 'INTERVENÇÃO AFETADA',
       'Descrição', 'km(i)', 'km(f)', 'Ext.(km)', 'FINANCEIRO(R$)',
       'PER\nANO(Execução)', 'LP', 'LI', 'LO', 'DATA DE INÍCIO', 'SINAFLOR',
       'SISGLAF', 'SEI', 'Risco Principal', 'CRITICIDADE', 'SITUAÇÃO GERAL',
       'COMENTÁRIOS GERAIS', 'Data do Atendimento', 'NÍVEL DE ALERTA']]

In [136]:
#df_riscos_copy.to_excel('Planilha Monitoramento de Riscos_Final.xlsx', index=False)

---

<h1><b><i>ID - RISCO</b></i></h1>
Sheet


In [137]:
df_id_riscos = df_riscos_copy[df_riscos_copy['Bin'] == 1]

In [138]:
#df_id_riscos = pd.read_excel('Planilha Monitoramento de Riscos_Final.xlsx')

In [139]:
(df_id_riscos['Bin'] == 1).sum()

113

In [140]:
df_id_riscos = df_id_riscos.reset_index(drop=True)

In [141]:
df_id_riscos.index = range(1, len(df_id_riscos) + 1)

In [142]:
df_id_riscos.to_excel('RISCOS - LIMPO\ID - RISCOS.xlsx')

<>:1: SyntaxWarning: invalid escape sequence '\I'
<>:1: SyntaxWarning: invalid escape sequence '\I'
C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_8868\941979350.py:1: SyntaxWarning: invalid escape sequence '\I'
  df_id_riscos.to_excel('RISCOS - LIMPO\ID - RISCOS.xlsx')


<h3><b><i>'Descrição'</b></i></h3>
Planilha 'Descrição'


In [143]:
df_desc = pd.read_excel('RISCOS - LIMPO\ID - RISCOS.xlsx', sheet_name='Sheet1',
    usecols=['COMENTÁRIOS GERAIS'])

<>:1: SyntaxWarning: invalid escape sequence '\I'
<>:1: SyntaxWarning: invalid escape sequence '\I'
C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_8868\498207847.py:1: SyntaxWarning: invalid escape sequence '\I'
  df_desc = pd.read_excel('RISCOS - LIMPO\ID - RISCOS.xlsx', sheet_name='Sheet1',


In [144]:
df_desc = pd.read_excel('RISCOS - LIMPO\ID - RISCOS.xlsx', sheet_name='Sheet1',
    usecols=['COMENTÁRIOS GERAIS'])

<>:1: SyntaxWarning: invalid escape sequence '\I'
<>:1: SyntaxWarning: invalid escape sequence '\I'
C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_8868\498207847.py:1: SyntaxWarning: invalid escape sequence '\I'
  df_desc = pd.read_excel('RISCOS - LIMPO\ID - RISCOS.xlsx', sheet_name='Sheet1',


In [145]:
df_desc_copy = df_desc.copy()

In [146]:
df_desc_copy['COMENTÁRIOS GERAIS'].unique()

array(['Emitida em 26/10/2023', 'Emitida em 12/09/2023',
       'Emitida em 05/12/2023', 'Em análise desde 02/04/2024 - Eugênio',
       'Rejeitado no sistema desde 11/01/2024. Não localizada nova demanda no sistema',
       'Em análise desde 17/04/2024 - Eugênio', 'Emitida em 05/09/2023',
       'Emitida em 29/02/2024', 'Emitida LI 1478/2023 em 02/10/2023',
       'Emitida em 15/10/2023',
       'Distribuído para análise. Empresa solicitou retificação logo após emissão da ASV original. Em análise desde 06/03/24 - Lívia',
       'Processo tramitado via SISGLAF 056.446, LI retificada em 21/12/2023',
       'Emitida em 17/09/2024',
       'Recibo de ASV devolvido ao interessado. Protocolado inventário florestal complementar. Aguardando distribuição desde 15/06/2024 - Rô',
       'Recibo de ASV devolvido ao interessado. Protocolado inventário florestal complementar. Aguara homologação desde 06/09/2024 - ASV emitida em 26/09/2024',
       'Provável registro duplicado',
       'ASV bloquead

In [147]:
# Inicializa as colunas para cada categoria, incluindo uma para "Outras Descrições"
categorias = ['Emitida', 'Em análise', 'Aguardando', 'Solicitada', 'Sem informação', 'Pendência', 'Outras Descrições']
for categoria in categorias:
    df_desc_copy[categoria] = None

# Iterando sobre as linhas do DataFrame
for index, row in df_desc_copy.iterrows():
    texto = row['COMENTÁRIOS GERAIS']
    if pd.isna(texto):  # Se a descrição for NaN, pula para a próxima iteração
        continue

    texto_inicial = texto.split()[0].lower()  # Considera apenas a primeira palavra para a categorização
    
    # Inicialmente classifica como "Outras Descrições"
    categoria = 'Outras Descrições'

    # Verifica o início da descrição para determinar a categoria
    if texto_inicial.startswith('emitida'):
        categoria = 'Emitida'
    elif texto_inicial.startswith('em') and 'análise' in texto.lower():
        categoria = 'Em análise'
    elif texto_inicial.startswith('aguardando'):
        categoria = 'Aguardando'
    elif texto_inicial.startswith('solicitada'):
        categoria = 'Solicitada'
    elif texto_inicial.startswith('sem') and 'informação' in texto.lower():
        categoria = 'Sem informação'
    elif texto_inicial.startswith('pendência'):
        categoria = 'Pendência'
    
    # Atribui a descrição à categoria identificada
    df_desc_copy.at[index, categoria] = texto

In [148]:
df_desc_copy

,COMENTÁRIOS GERAIS,Emitida,Em análise,Aguardando,Solicitada,Sem informação,Pendência,Outras Descrições
0,Emitida em 26/10/2023,Emitida em 26/10/2023,None,None,None,None,None,None
1,Emitida em 12/09/2023,Emitida em 12/09/2023,None,None,None,None,None,None
2,Emitida em 05/12/2023,Emitida em 05/12/2023,None,None,None,None,None,None
3,Emitida em 05/12/2023,Emitida em 05/12/2023,None,None,None,None,None,None
4,Em análise desde 02/04/2024 - Eugênio,None,Em análise desde 02/04/2024 - Eugênio,None,None,None,None,None
...,...,...,...,...,...,...,...,...
108,Em análise desde 02/04/2024 - Lívia,None,Em análise desde 02/04/2024 - Lívia,None,None,None,None,None
109,Solicitação de Retificação de Licença - 084426...,None,None,None,None,None,None,Solicitação de Retificação de Licença - 084426...
110,SISGLAF 060.578 – Parecer da Lívia prestes a s...,None,None,None,None,None,None,SISGLAF 060.578 – Parecer da Lívia prestes a s...
111,10119193 – Pendência acusada pelo próprio soli...,None,None,None,None,None,None,10119193 – Pendência acusada pelo próprio soli...


In [149]:
df_desc_copy = df_desc_copy.drop(columns=['COMENTÁRIOS GERAIS'])

In [150]:
df_desc_copy.to_excel('RISCOS - LIMPO\Descrições - Risco.xlsx', index=False)

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_8868\1304174817.py:1: SyntaxWarning: invalid escape sequence '\D'
  df_desc_copy.to_excel('RISCOS - LIMPO\Descrições - Risco.xlsx', index=False)


<h2><b><i>'ID-RISCO' + 'Descrição'</b></i></h2>
Script


In [151]:
import pandas as pd

In [152]:
df_riscos_concat = pd.read_excel('RISCOS - LIMPO\ID - RISCOS.xlsx')
df_desc_concat = pd.read_excel('RISCOS - LIMPO\Descrições - Risco.xlsx')

<>:1: SyntaxWarning: invalid escape sequence '\I'
<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\I'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_8868\1534851765.py:1: SyntaxWarning: invalid escape sequence '\I'
  df_riscos_concat = pd.read_excel('RISCOS - LIMPO\ID - RISCOS.xlsx')
C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_8868\1534851765.py:2: SyntaxWarning: invalid escape sequence '\D'
  df_desc_concat = pd.read_excel('RISCOS - LIMPO\Descrições - Risco.xlsx')


In [153]:
df_desc_concat.fillna(0, inplace=True)

In [154]:
df_final = pd.concat([df_riscos_concat, df_desc_concat], axis=1)

In [155]:
df_final

,Unnamed: 0,SETOR,UF,BR,Bin,EMPREENDIMENTO,EXECUTOR\n(Grupo Controlador),DATA DE INÍCIO DA CONCESSÃO,ANO DA CONCESSÃO,INTERFERÊNCIAS,...,COMENTÁRIOS GERAIS,Data do Atendimento,NÍVEL DE ALERTA,Emitida,Em análise,Aguardando,Solicitada,Sem informação,Pendência,Outras Descrições
0,1,Rodoviário,RS,101,1,VIA SUL,CCR,2019-02-15,6,AMBIENTAL,...,Emitida em 26/10/2023,2023-10-26,2,Emitida em 26/10/2023,0,0.0,0,0,0,0
1,2,Rodoviário,RS,101,1,VIA SUL,CCR,2019-02-15,6,AMBIENTAL,...,Emitida em 12/09/2023,2023-09-12,2,Emitida em 12/09/2023,0,0.0,0,0,0,0
2,3,Rodoviário,RS,101,1,VIA SUL,CCR,2019-02-15,6,AMBIENTAL,...,Emitida em 05/12/2023,2023-12-05,2,Emitida em 05/12/2023,0,0.0,0,0,0,0
3,4,Rodoviário,RS,101,1,VIA SUL,CCR,2019-02-15,6,AMBIENTAL,...,Emitida em 05/12/2023,2023-12-05,2,Emitida em 05/12/2023,0,0.0,0,0,0,0
4,5,Rodoviário,RS,101,1,VIA SUL,CCR,2019-02-15,6,AMBIENTAL,...,Em análise desde 02/04/2024 - Eugênio,NaT,1,0,Em análise desde 02/04/2024 - Eugênio,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,109,Rodoviário,MS,163,1,MSVIAS,CCR,2014-04-22,10,AMBIENTAL,...,Em análise desde 02/04/2024 - Lívia,NaT,1,0,Em análise desde 02/04/2024 - Lívia,0.0,0,0,0,0
109,110,Rodoviário,MS,163,1,MSVIAS,CCR,2014-04-22,10,AMBIENTAL,...,Solicitação de Retificação de Licença - 084426...,NaT,1,0,0,0.0,0,0,0,Solicitação de Retificação de Licença - 084426...
110,111,Rodoviário,MS,163,1,MSVIAS,CCR,2014-04-22,10,AMBIENTAL,...,SISGLAF 060.578 – Parecer da Lívia prestes a s...,NaT,2,0,0,0.0,0,0,0,SISGLAF 060.578 – Parecer da Lívia prestes a s...
111,112,Rodoviário,MS,163,1,MSVIAS,CCR,2014-04-22,10,AMBIENTAL,...,10119193 – Pendência acusada pelo próprio soli...,NaT,1,0,0,0.0,0,0,0,10119193 – Pendência acusada pelo próprio soli...


In [156]:
df_final.rename(columns={'Unnamed: 0': 'ID-ÚNICO'}, inplace=True)

In [157]:
df_final

,ID-ÚNICO,SETOR,UF,BR,Bin,EMPREENDIMENTO,EXECUTOR\n(Grupo Controlador),DATA DE INÍCIO DA CONCESSÃO,ANO DA CONCESSÃO,INTERFERÊNCIAS,...,COMENTÁRIOS GERAIS,Data do Atendimento,NÍVEL DE ALERTA,Emitida,Em análise,Aguardando,Solicitada,Sem informação,Pendência,Outras Descrições
0,1,Rodoviário,RS,101,1,VIA SUL,CCR,2019-02-15,6,AMBIENTAL,...,Emitida em 26/10/2023,2023-10-26,2,Emitida em 26/10/2023,0,0.0,0,0,0,0
1,2,Rodoviário,RS,101,1,VIA SUL,CCR,2019-02-15,6,AMBIENTAL,...,Emitida em 12/09/2023,2023-09-12,2,Emitida em 12/09/2023,0,0.0,0,0,0,0
2,3,Rodoviário,RS,101,1,VIA SUL,CCR,2019-02-15,6,AMBIENTAL,...,Emitida em 05/12/2023,2023-12-05,2,Emitida em 05/12/2023,0,0.0,0,0,0,0
3,4,Rodoviário,RS,101,1,VIA SUL,CCR,2019-02-15,6,AMBIENTAL,...,Emitida em 05/12/2023,2023-12-05,2,Emitida em 05/12/2023,0,0.0,0,0,0,0
4,5,Rodoviário,RS,101,1,VIA SUL,CCR,2019-02-15,6,AMBIENTAL,...,Em análise desde 02/04/2024 - Eugênio,NaT,1,0,Em análise desde 02/04/2024 - Eugênio,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,109,Rodoviário,MS,163,1,MSVIAS,CCR,2014-04-22,10,AMBIENTAL,...,Em análise desde 02/04/2024 - Lívia,NaT,1,0,Em análise desde 02/04/2024 - Lívia,0.0,0,0,0,0
109,110,Rodoviário,MS,163,1,MSVIAS,CCR,2014-04-22,10,AMBIENTAL,...,Solicitação de Retificação de Licença - 084426...,NaT,1,0,0,0.0,0,0,0,Solicitação de Retificação de Licença - 084426...
110,111,Rodoviário,MS,163,1,MSVIAS,CCR,2014-04-22,10,AMBIENTAL,...,SISGLAF 060.578 – Parecer da Lívia prestes a s...,NaT,2,0,0,0.0,0,0,0,SISGLAF 060.578 – Parecer da Lívia prestes a s...
111,112,Rodoviário,MS,163,1,MSVIAS,CCR,2014-04-22,10,AMBIENTAL,...,10119193 – Pendência acusada pelo próprio soli...,NaT,1,0,0,0.0,0,0,0,10119193 – Pendência acusada pelo próprio soli...


<h1><b><i>Coluna FIN e KM novas</b></i></h1>
Sem Duplicadas.


In [158]:
def remove_duplicated_values(df):
    # Define as colunas para identificar duplicatas
    cols_to_check = ['INTERFERÊNCIAS', 'INTERVENÇÃO AFETADA', 'Descrição', 'km(i)', 'km(f)']
    
    # Marcar duplicatas, mantendo a primeira ocorrência de cada grupo
    duplicates_mask = df.duplicated(subset=cols_to_check, keep='first')
    
    # Criando as novas colunas 'Ext.(km)!' e 'FINANCEIRO(R$)!' onde não houver duplicatas
    df['Ext.(km)!'] = df['Ext.(km)'].where(~duplicates_mask, None)
    df['FINANCEIRO(R$)!'] = df['FINANCEIRO(R$)'].where(~duplicates_mask, None)
    
    return df

In [159]:
# Aplicar a função ao DataFrame
df_final = remove_duplicated_values(df_final)

In [160]:
# Inserir as novas colunas imediatamente após as colunas originais
idx_ext_km = df_final.columns.get_loc('Ext.(km)') + 1
idx_fin = df_final.columns.get_loc('FINANCEIRO(R$)') + 1

In [161]:
df_final.insert(idx_ext_km, 'Ext.(km)!', df_final.pop('Ext.(km)!'))
df_final.insert(idx_fin, 'FINANCEIRO(R$)!', df_final.pop('FINANCEIRO(R$)!'))

In [162]:
# Salvar o DataFrame atualizado de volta ao arquivo Excel original
df_final.to_excel('RISCOS - LIMPO\Riscos - BI.xlsx', index=False, sheet_name= 'RISCOS')

<>:2: SyntaxWarning: invalid escape sequence '\R'
<>:2: SyntaxWarning: invalid escape sequence '\R'
C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_8868\1613408082.py:2: SyntaxWarning: invalid escape sequence '\R'
  df_final.to_excel('RISCOS - LIMPO\Riscos - BI.xlsx', index=False, sheet_name= 'RISCOS')
